In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4
# pip install -qU langchain-openai

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass('OPENAI_API_KEY: ')

# import dotenv

# dotenv.load_dotenv()


In [ ]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('LANGCHAIN_API_KEY: ')

In [ ]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llamafamily/llama3-chinese-8b-instruct",temperature=0.1,top_p=0.3)
# llm.invoke("台灣是中國的嗎?")

In [ ]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 1. Load, chunk and index the contents of the blog to create a retriever.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )

loader = TextLoader("C:\\Users\\User\\Downloads\\九大體質.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=Ollama(model="llamafamily/llama3-chinese-8b-instruct"))
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "台灣是中國的嗎?"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。中醫十問是哪十問?"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。中醫十問的第三問是什麼?(只要問診問題)"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。中醫十問的第三問是什麼?請稍微描述一下問診細節。"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。判斷以下描述最接近中醫十問的哪一問?\n- - -\n＂我經常失眠，半夜會被驚醒＂"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。判斷以下描述最接近中醫十問的哪一{問診問題}?\n- - -\n＂我經常便祕，大便一顆一顆的像羊便便一樣，很乾很硬＂"})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。問診問題A:大便與問診問題B:睡眠，是完全相同的一個問診問題嗎"})
response["answer"]

In [ ]:
llm.invoke("""範例：當問診問題是「口渴」時，使用者應該回答：「我常常會工作到忘記喝水，一天可能只喝1200ml左右。」
請先分析使用者的回答屬於中醫十問中的哪一問，接著再比對是否有正確回答問診問題。例如：使用者的回答屬於「口渴」，與問診問題「口渴」範圍一致。
請進行問診測試，回答以下問題：「使用者是否正確理解並回答了問題？」請回答「是，使用者正確理解並回答了問題」或「否，雖然使用者有回答，但答非所問」。
以下是測試內容：
- - -
問診問題： 大便
使用者回答： 我經常失眠，半夜會被驚醒。

- - -
請一步一步的詳細分析，這對我的事業很重要，如果做得好我會給你小費。
""")

In [ ]:
response = rag_chain.invoke({"input": """請一步一步的詳細分析，這對我的事業很重要，如果做得好我會給你小費。
範例：當問診問題是「口渴」時，使用者應該回答：「我常常會工作到忘記喝水，一天可能只喝1200ml左右。」
請先分析使用者的回答屬於中醫十問中的哪一問，接著再比對是否有正確回答問診問題。例如：使用者的回答屬於「口渴」，與問診問題「口渴」範圍一致。
請進行問診測試，回答以下問題：「使用者是否正確理解並回答了問題？」請回答「是，使用者正確理解並回答了問題」或「否，雖然使用者有回答，但答非所問」。
以下是測試內容：
- - -
問診問題： 大便
使用者回答： 我經常失眠，半夜會被驚醒。
"""})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": """請一步一步的詳細分析，這對我的事業很重要，如果做得好我會給你小費。
範例：當問診問題「口渴」，使用者回答：「我的尿是深黃色的，大概五個小時才小便一次。」
請先分析使用者的回答屬於中醫十問中的哪一問，接著再比對是否有正確回答問診問題。例如：使用者的回答屬於「小便」，與問診問題「口渴」範圍不一致。
請進行問診測試，回答以下問題：「使用者是否正確理解並回答了問題？」請回答「是，使用者正確理解並回答了問題」或「否，雖然使用者有回答，但答非所問」。
以下是測試內容，請回答我：
- - -
問診問題： 大便
使用者回答： 我經常失眠，半夜會被驚醒。
"""})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": """請一步一步的詳細分析，這對我的事業很重要，如果做得好我會給你小費。
範例：當問診問題「口渴」，使用者回答：「我的尿是深黃色的，大概五個小時才小便一次。」
請先分析使用者的回答屬於中醫十問中的哪一問，接著再比對是否有正確回答問診問題。例如：使用者的回答屬於「小便」，與問診問題「口渴」範圍不一致。
請進行問診測試，回答以下問題：「使用者是否正確理解並回答了問題？」請回答「是，使用者正確理解並回答了問題」或「否，雖然使用者有回答，但答非所問」。
以下是測試內容，請回答我：
- - -
問診問題： 大便
使用者回答： 我經常便祕，大便一顆一顆的像羊便便一樣，很乾很硬
"""})
response["answer"]

In [ ]:

response = rag_chain.invoke({"input": """請一步一步的詳細分析，這對我的事業很重要，如果做得好我會給你小費。
當問診問題是「問題」時，請按照以下步驟確認使用者的回答是否正確：
1. 確認問診問題：問題。
2. 確認使用者回答：回答。
3. 分析使用者的回答屬於中醫十問中的哪一問：
中醫十問包括：睡眠、胃口、大便、小便、口渴、寒熱、汗、體力、性功能、女子月經
確認回答屬於哪一問。
4. 比對問診問題與使用者回答：
確認問診問題屬於哪一問。
確認使用者回答屬於哪一問。
5. 確認使用者是否正確回答了問診問題：
如果問診問題與使用者回答屬於同一問，回答：「是，使用者正確理解並回答了問題。」
如果問診問題與使用者回答不屬於同一問，回答：「否，雖然使用者有回答，但答非所問。」
請回答以下問題：「使用者是否正確理解並回答了問題？」

以下是範例：
問診細節： 你很渴嗎？如渴，最想喝什麼溫度的水？如不渴，時常會忘記喝水嗎？還是再怎麼喝也不能止渴呢？
使用者回答： 我常常會工作到忘記喝水，一天可能只喝1200ml左右。

分析：

1. 確認問診問題：口渴。
2. 確認使用者回答：我常常會工作到忘記喝水，一天可能只喝1200ml左右。
3. 分析使用者的回答屬於中醫十問中的哪一問：
使用者的回答屬於「口渴」這一問。
4. 比對問診問題與使用者回答：
問診問題是關於「口渴」。
使用者的回答是關於「口渴」。
5. 確認使用者是否正確回答了問診問題：
是，使用者正確理解並回答了問題。

- - -
我們正式開始：
問診細節： 你便秘嗎？每天有大便嗎？大便顏色是什麼？是下利嗎？很臭還是無味？等等。
使用者回答： 我經常失眠，半夜會被驚醒。
"""})
response["answer"]

In [ ]:
response = rag_chain.invoke({"input": "根據文件。請簡單分析裡面文件內有提到體質嗎?如果有，提到幾種體質?具體是哪幾個?"})
response["answer"]

In [ ]:
response = rag_chain.invoke({
    "input":"根據文件。請分析文件中有提到以下描述嗎?如果沒有請回答不知道，如果有，這是哪一種體質?\n- - -\n常見表現特徵為手腳冰冷、臉色蒼白、身體畏寒、喜歡吃溫熱飲食。容易發生水腫、腹瀉大便不成形，易表現出精神不濟，容易感冒。體態上呈現肌肉鬆軟不結實，舌頭胖大、舌苔變厚呈白色水滑。"})
response["answer"]

In [1]:
import os
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

model_name = "llamafamily/llama3-chinese-8b-instruct"
file_path = "C:\\Users\\User\\Documents\\work\\tcm-agent\\data\\txt"

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise. Answer in Chinese."
    "\n\n"
    "{context}"
)

llm = Ollama(model=model_name,temperature=0.2,top_p=0.3)
embedding= OllamaEmbeddings(model=model_name,temperature=0.2,top_p=0.3)
# loader = TextLoader("C:\\Users\\User\\Downloads\\中醫體質與問診.txt")
# loader = TextLoader("C:\\Users\\User\\Downloads\\九大體質.txt")


class RagChain:
    def __init__(self,llm, embedding,file_name,system_prompt) -> None:
        self.llm = llm
        self.embedding = embedding
        # self.system_prompt = system_prompt
        
        total_path = os.path.normpath(f"{file_path}/{file_name}")
        self.loader = TextLoader(total_path,encoding='utf-8')
        self.docs = self.loader.load()
        
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.splits = self.text_splitter.split_documents(self.docs)
        self.vector_store = Chroma.from_documents(documents= self.splits, embedding= embedding)
        self.retriever = self.vector_store.as_retriever()
        
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        
        self.question_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        self.rag_chain = create_retrieval_chain(self.retriever, self.question_answer_chain)
            
    def __call__(self, input_data):
        return self.rag_chain.invoke(input_data)

    
# 中醫十問
zhongyi_shiwen = RagChain(llm, embedding, "中醫十問.txt", system_prompt)   
# 九大體質
jiuda_tizhi = RagChain(llm, embedding, "九大體質.txt", system_prompt) 
self_introduction = RagChain(llm, embedding, "自我介紹.txt", system_prompt)    


In [15]:

SongBieYu = RagChain(llm, embedding, "送別語.txt", system_prompt)  

response_4 = SongBieYu({"input": "based on knowledge。參考文件內容說再見，完整一點的版本"})
print(response_4['answer'])

很高興為您服務，中醫提倡“治標治本”，除了使用藥物治療，還要通過調整生活方式來達到長期的健康目標。這也是中醫強調“治未病”的理念，即預防勝於治療，通過健康的生活方式來防止疾病的發生。需要注意的是，中藥治療可能會有一些副作用，如果感到非常不適，請立即就醫或尋求專業幫助。祝平安喜樂，再見！


In [11]:
response_1 = self_introduction({"input": "based on knowledge。請按照＂自我介紹.txt＂文件做自我介紹"})
print(response_1['answer'])

您好，我是經方派虛擬中醫師。我們的治療方法根據您的症狀來開出適合的處方。在治療過程中，可能會出現一些反應，但這是正常的。請記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，以便我們能夠為您提供最好的治療。此外，我們提倡“治標治本”，除了使用藥物治療，更重要的是通過調整生活方式來達到長期的健康目標。這也是中醫強調“治未病”的理念，即預防勝於治療，通過健康的生活方式來防止疾病的發生。需要注意的是，中藥治療可能會有一些副作用，如果感到非常不適，請立即就醫或尋求專業幫助。謝謝！


In [ ]:

response_2 = zhongyi_shiwen({"input": "based on knowledge。請分析文件，中醫十問是哪十問?請告訴我問診名稱就好。"})
print(response_2['answer'])


In [ ]:


response_3 = jiuda_tizhi({"input": "based on knowledge。請分析文件中有提到以下描述嗎?如果沒有請回答不知道，如果有，這是哪一種體質?\n- - -\n常見表現特徵為手腳冰冷、臉色蒼白、身體畏寒、喜歡吃溫熱飲食。容易發生水腫、腹瀉大便不成形，易表現出精神不濟，容易感冒。體態上呈現肌肉鬆軟不結實，舌頭胖大、舌苔變厚呈白色水滑。"})
print(response_3['answer'])


In [ ]:
response_4 = jiuda_tizhi({"input": "based on knowledge。如果沒有請回答不知道，如果有，請詳細說明以下體質有什麼病症和飲食建議\n- - -\n血瘀質"})
print(response_4['answer'])


In [ ]:

response_5 = zhongyi_shiwen({"input": "based on knowledge。中醫十問當中的第一問的問診細節是什麼?只要問診細節：後面的"})
print(response_5['answer'])


In [ ]:
tttt = llm.invoke(f"請把以下兩句話請按照順序進行合併。第一句：{response_1['answer']}\n 接著讓我們來開始第一個問題 \n第二句：{response_5['answer']}")
print(tttt)

In [ ]:
import json

class Agent:
    sex = 'male'
    messages = []
    _qa = ["睡眠","胃口","大便","小便","口渴","寒熱","汗","體力","性功能","女子月經"]
    questions = {}
    answers = {}
    question_count = 0
    def __init__(self,sex = 'male') -> None:
        self.sex = sex
        pass
    
    def invoke(self,user_input):
        if self.question_count == 0:
            self_introduction_response = self_introduction({"input": "based on knowledge。請按照＂自我介紹.txt＂文件做自我介紹"})
            zhongyi_shiwen_response = zhongyi_shiwen({"input": "based on knowledge。中醫十問當中的第一問的問診細節是什麼?只要問診細節：後面的文字就好"})
            question_key = self._qa[self.question_count]
            self.questions[question_key] = zhongyi_shiwen_response['answer']
            
            response = llm.invoke(f"請把以下兩句話請按照順序進行合併。第一句：{self_introduction_response['answer']}\n 接著讓我們來開始第一個問題 \n第二句：{zhongyi_shiwen_response['answer']}")
        elif (self.sex == 'male' and self.question_count >= 9) or (self.sex == 'female' and self.question_count >= 10 ):

            # 將過濾後的字典轉換為 JSON 字串
            # json_string = json.dumps(self.answers, ensure_ascii=False, indent=4)
            formatted_list = [f"問{key}，答 {value}" for key, value in self.answers.items()]
            formatted_string = "。".join(formatted_list)
            # llm_merge_response =  llm.invoke(f"請把以資料請按照順序進行合併，並按照格式表示＂問key，答value＂。以下是資料：\n{json_string}")
            jiuda_tizhi_response = jiuda_tizhi({"input": f"based on knowledge。請分析文件，以下描述最接近哪幾種體質特徵(1~3種)?有什麼飲食上的建議?\n- - -\n{formatted_string}"})
            # llm_organizing_response =  llm.invoke(f"請把以資料重新整理(先描述體質種類，最後才是給飲食建議)、合併重複內容。以下是資料：\n{jiuda_tizhi_response['answer']}")
            response = jiuda_tizhi_response['answer']
        else:
            answer_key = self._qa[self.question_count-1]
            self.answers[answer_key] = user_input
            
            key = self._qa[self.question_count]
            zhongyi_shiwen_response = zhongyi_shiwen({"input": f"based on knowledge。中醫十問當中的{key}的問診細節是什麼?只要問診細節：後面的的文字就好"})
            
            question_key = self._qa[self.question_count]
            self.questions[question_key] = zhongyi_shiwen_response['answer']
            
            llm_response =  llm.invoke(f"如果以下文字內容是正面的，請你回答＂好的，我知道了！＂；如果以下文字內容是負面的，請總結以下文字大概意思就好字數不超過二十字，並在最前面加上＂我了解你有(_總結後的文字_)的問題。＂。\n{user_input}")
            response = llm.invoke(f"請把以下幾段話請按照順序進行合併。 {llm_response} \n　接著讓我們來進行下一個問題 \n {zhongyi_shiwen_response['answer']}")
            
        
        
        self.question_count += 1
        self.messages.append(response)
        print(response)
        return response

In [ ]:
agent = Agent(sex='female')
agent.invoke('我要開始問診')
agent.invoke('我可以正常入睡，但半夜常常會莫名起來，然後就睡不回去了')
agent.invoke('我最近都沒什麼胃口吃不下飯，只想吃甜食和炸物')
agent.invoke('最近腹瀉吃什麼馬上就會拉肚子')
agent.invoke('小便是深黃色，大概間隔四五個小時久久才會去尿一次')
agent.invoke('喜歡喝冰水，但是會因為忙到忘記喝水，一天可能才喝1200c.c.的水量')
agent.invoke('覺得四肢冰冷')
agent.invoke('半夜會盜汗')
agent.invoke('一直覺得很疲憊，怎麼睡都不夠，早上都爬不起來')
agent.invoke('性功能一切正常')
agent.invoke('月經不規律，每一次來都很痛，血算是偏多')

In [ ]:
agent = Agent(sex='male')
agent.invoke('我要開始問診')
agent.invoke('我因為工作的關係常常熬夜，但是可以一覺到天亮')
agent.invoke('我最近都沒什麼胃口吃不下飯')
agent.invoke('最近便祕，大概兩三天才會大出一點點，像羊便便一樣一顆一顆，又乾又硬')
agent.invoke('小便是透明的，大概半個小時就會上一次廁所')
agent.invoke('喜歡喝冰水，一天可以喝到2200c.c.的水量')
agent.invoke('覺得四肢很熱')
agent.invoke('平常待在冷氣房不怎麼流汗，但是只要一離開冷氣房就會一直流汗')
agent.invoke('一直覺得很疲憊，怎麼睡都不夠')
agent.invoke('性無能，勃起有困難')

In [ ]:
agent = Agent(sex='female')
agent.invoke('我要開始問診')

In [ ]:
agent.invoke('我常常失眠，好不容易睡著卻很容易被一點聲音嚇醒')

In [ ]:
print(f"questions:\n{agent.questions}\n")
print(f"answers:\n{agent.answers}\n")
print(f"messages:\n{agent.messages}\n")

In [ ]:
# 如果想要重新回答上一題
agent.question_count -= 1

In [ ]:
print(f"agent.question_count: {agent.question_count}")

In [ ]:
agent.invoke('我最近都沒什麼胃口吃不下飯')

In [ ]:
agent.invoke('最近便祕，大概兩三天才會大出一點點，像羊便便一樣一顆一顆，又乾又硬')

In [ ]:
agent.invoke('小便是深黃色，大概四五個小時才會上一次廁所')

In [ ]:
agent.invoke('喜歡喝冰水，但是會因為忙到忘記喝水，一天可能才喝1200c.c.的水量')

In [ ]:
agent.invoke('覺得四肢很熱')

In [ ]:
agent.invoke('平常待在冷氣房不怎麼流汗，但是只要一離開冷氣房就會一直流汗')

In [ ]:
agent.invoke('一直覺得很疲憊，怎麼睡都不夠')

In [ ]:
agent.invoke('性功能一切正常')

In [ ]:
agent.invoke('月經不規律，每一次來都很痛，血算是偏多')

In [ ]:

from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END, MessageGraph, Graph
from langgraph.prebuilt import ToolNode
from langchain_community.llms import Ollama
# assign AgentState as an empty dict
# AgentState = {}
# # messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
# AgentState["messages"] = []

model = Ollama(model="llamafamily/llama3-chinese-8b-instruct",temperature=0.2)


def collectUserAnswer(state):
    """判斷使用者目前的答案屬於中醫十問的哪一個問診問題"""
    print('collectUserAnswer: '+state)
    complete_query = ''
    # messages = state['messages']
    # user_input = messages[-1]
    if question_count == 0:
        complete_query = f"你是是經方派中醫師。問診對我們非常重要。我會根據您的回答，開出適合的處方。治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。請用五十個字簡單自我介紹" 
    else:
        complete_query = f"你的任務是判斷以下描述最接近中醫十問的哪一問診問題。不要做多餘的事情，只要問診問題的名稱。\n- - -\n以下是一段描述：{state}" 
    response = rag_chain.invoke({"input":complete_query})
    # print(response)
    # state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return response["answer"]

def collectUserAnswer(state):
    """判斷使用者目前的答案屬於中醫十問的哪一個問診問題"""
    print('collectUserAnswer: '+state)
    # messages = state['messages']
    # user_input = messages[-1]
    complete_query = f"你的任務是判斷以下描述最接近中醫十問的哪一問診問題。不要做多餘的事情，只要問診問題的名稱。\n- - -\n以下是一段描述：{state}" 
    response = rag_chain.invoke({"input":complete_query})
    # print(response)
    # state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return response["answer"]

def function_2(state):
    print('function_2: '+state+'\n\n')
    return state

# model = ChatOpenAI(temperature=0)
# model_with_tools = llm.bind_tools(tools=[multiply])
workflow = Graph()

workflow.add_node("collector", collectUserAnswer)
workflow.add_node("tool", function_2)

workflow.add_edge('collector', 'tool')

# workflow.add_node("multiply", tool_node)

workflow.set_entry_point("collector")
workflow.set_finish_point("tool")

runnable = workflow.compile()

In [ ]:
from IPython.display import Image, display
try:
    display(Image(runnable.get_graph(xray=True).draw_mermaid_png()))
except:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
# inputs = {
#     "messages": ["我經常失眠，半夜會被驚醒。"]
# }
# runnable.invoke("我經常失眠，半夜會被驚醒。")
# runnable.invoke("我便秘很嚴重，兩三天才便一次，像羊便便一樣一顆一顆的又乾又硬")

runnable.invoke("我胃口不好，吃不下飯，但喜歡吃甜食和炸物。")